In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.7/699.7 kB 3.9 MB/s eta 0:00:00


# prepare data

## data ban đầu

In [8]:
#https://drive.google.com/file/d/1Qq5PLe0ntyxYKSKZecVwTt9TaaQ-lL8A/view?usp=sharing
!curl -L -o 'data1.zip' 'https://drive.usercontent.google.com/download?id=1Qq5PLe0ntyxYKSKZecVwTt9TaaQ-lL8A&export=download&authuser=1&confirm=t'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.0G  100 17.0G    0     0  96.5M      0  0:03:00  0:03:00 --:--:-- 95.3M


In [9]:
#https://drive.google.com/file/d/18J2DIDUX2dvX3_pBGWlbF4Dgi2I7O3Qm/view?usp=sharing
!curl -L -o 'label1.csv' 'https://drive.usercontent.google.com/download?id=18J2DIDUX2dvX3_pBGWlbF4Dgi2I7O3Qm&export=download&authuser=1&confirm=t'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2260k  100 2260k    0     0  2668k      0 --:--:-- --:--:-- --:--:-- 2666k


## data bonus

In [10]:
#https://drive.google.com/file/d/1mgRpPyh2GzdFl2ZsrEOJfIhYx-aBiOoT/view?usp=sharing
!curl -L -o 'data2.zip' 'https://drive.usercontent.google.com/download?id=1mgRpPyh2GzdFl2ZsrEOJfIhYx-aBiOoT&export=download&authuser=1&confirm=t'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.9G  100 13.9G    0     0  97.6M      0  0:02:26  0:02:26 --:--:-- 84.0M


In [11]:
#https://drive.google.com/file/d/14HkR-x1445_mE7ipfqsm5nrzlHFIqOZR/view?usp=sharing
!curl -L -o 'label2.csv' 'https://drive.usercontent.google.com/download?id=14HkR-x1445_mE7ipfqsm5nrzlHFIqOZR&export=download&authuser=1&confirm=t'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1946k  100 1946k    0     0  3464k      0 --:--:-- --:--:-- --:--:-- 3469k


## gộp 2 data

In [14]:
%cd /content/
!unzip -qq data1.zip
!unzip -qq data2.zip

/content
mnt/md0/projects/sami-hackathon/private/data/82879374.jpg:  write error (disk full?).  Continue? (y/n/^C) Y

replace data/85391473.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A

data/85391473.jpg:  write error (disk full?).  Continue? (y/n/^C) y

data/59454118.jpg:  write error (disk full?).  Continue? (y/n/^C) A



In [13]:
import os
import shutil
img_data1_path='mnt/md0/projects/sami-hackathon/private/data'
img_data2_path='data'
os.makedirs('all_img',exist_ok=True)
for file in os.listdir(img_data1_path):
  shutil.move(os.path.join(img_data1_path,file),'all_img')

for file in os.listdir(img_data2_path):
  shutil.move(os.path.join(img_data2_path,file),'all_img')

FileNotFoundError: [Errno 2] No such file or directory: 'mnt/md0/projects/sami-hackathon/private/data'

In [ ]:
import pandas as pd
data1=pd.read_csv('label1.csv')
data2=pd.read_csv('label2.csv')
data2['race'] = data2['race'].replace('Mongolid', 'Mongoloid')
data = pd.concat([data1, data2], axis=0)
data['id']=data.index
data.to_csv('labels.csv',index=False)

## chia train dev

In [ ]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
%cd /content/
DATA_OUT_DIR = 'data_yolo'

for set_ in ['train', 'dev']:
    for dir_ in [os.path.join(DATA_OUT_DIR, set_),
                 os.path.join(DATA_OUT_DIR, set_, 'images'),
                 os.path.join(DATA_OUT_DIR, set_, 'labels')]:
        if os.path.exists(dir_):
            shutil.rmtree(dir_)
        os.makedirs(dir_,exist_ok=True)

data = pd.read_csv('labels.csv')
data['id'] = data.index
file_names = data['file_name'].unique()
train_file_names, dev_file_names = train_test_split(file_names, test_size=0.2, random_state=1234)
train = data[data['file_name'].isin(train_file_names)]
dev= data[data['file_name'].isin(dev_file_names)]

print(train.shape)
print(dev.shape)

In [ ]:
src_path='all_img'
with open(os.path.join(DATA_OUT_DIR, 'train', 'labels', f"classes.txt"), 'a') as f:
  f.write('face')
for idx, row in train.iterrows():
    img_name=row['file_name']
    id=row['id']
    w_img,h_img=row['width'],row['height']
    bbox = row['bbox'].replace('[','').replace(']','').replace(' ','').strip().split(',')
    x1, y1, x2, y2 = float(bbox[0]), float(bbox[1]), float(bbox[2])+ float(bbox[0]), float(bbox[1])+float(bbox[3])
    xc = (x1 + x2) / 2
    yc = (y1 + y2) / 2
    w = x2 - x1
    h = y2 - y1
    try:
      shutil.move(os.path.join(src_path,img_name),os.path.join(DATA_OUT_DIR,'train','images'))
    except:
      pass
    with open(os.path.join(DATA_OUT_DIR, 'train', 'labels', f"{img_name.replace('jpg','')}txt"), 'a') as f_ann:
      f_ann.write('0 {} {} {} {}\n'.format(xc/w_img, yc/h_img, w/w_img, h/h_img))
      f_ann.close()

In [ ]:
src_path='all_img'
with open(os.path.join(DATA_OUT_DIR, 'dev', 'labels', f"classes.txt"), 'a') as f:
  f.write('face')
for idx, row in dev.iterrows():
    img_name=row['file_name']
    id=row['id']
    w_img,h_img=row['width'],row['height']
    bbox = row['bbox'].replace('[','').replace(']','').replace(' ','').strip().split(',')
    x1, y1, x2, y2 = float(bbox[0]), float(bbox[1]), float(bbox[2])+ float(bbox[0]), float(bbox[1])+float(bbox[3])
    xc = (x1 + x2) / 2
    yc = (y1 + y2) / 2
    w = x2 - x1
    h = y2 - y1
    try:
      shutil.move(os.path.join(src_path,img_name),os.path.join(DATA_OUT_DIR,'dev','images'))
    except:
      pass
    with open(os.path.join(DATA_OUT_DIR, 'dev', 'labels', f"{img_name.replace('jpg','')}txt"), 'a') as f_ann:
      f_ann.write('0 {} {} {} {}\n'.format(xc/w_img, yc/h_img, w/w_img, h/h_img))
      f_ann.close()

# training

In [ ]:
%cd /content/
!rm data_yolo/mydataset.yaml
!echo 'train: /content/data_yolo/train' >> data_yolo/mydataset.yaml
!echo 'val: /content/data_yolo/dev' >> data_yolo/mydataset.yaml
!echo 'nc: 1' >> data_yolo/mydataset.yaml
!echo "names: ['face']" >> data_yolo/mydataset.yaml

/content


In [ ]:
!yolo task=detect mode=train model=yolov8m.pt data=data_yolo/mydataset.yaml epochs=20 imgsz=640 batch=16

Ultralytics YOLOv8.1.1 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=data_yolo/mydataset.yaml, epochs=20, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box